In [1]:
import pandas as pd
import numpy as np

#carga de datos
nombre_archivo = '../data/raw/aprehendidos_detenidos/dataset/mdi_detenidosaprehendidos_pm_2025_enero_octubre.xlsx'

try:
    print("Cargando archivo Excel... esto puede tardar unos segundos...")
    df_apre = pd.read_excel(
        nombre_archivo,
        sheet_name=1,
        dtype={'codigo_parroquia': str},  # proteger columna
        engine='openpyxl'
    )
    print(f"¡Cargado! Registros totales: {len(df_apre)}")

except Exception as e:
    print("Error cargando el Excel:", e)

# normalización de datos de latitud y longitud
for col in ["latitud", "longitud"]:
    df_apre[col] = (
        df_apre[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    df_apre[col] = pd.to_numeric(df_apre[col], errors="coerce")

# quitar filas sin coordenadas válidas
df_apre = df_apre.dropna(subset=["latitud", "longitud"])
df_apre = df_apre[(df_apre["latitud"] != 0) & (df_apre["longitud"] != 0)]


#nromalizar fecha y hora de detención/aprehensión
try:
    df_apre["fecha"] = pd.to_datetime(df_apre["fecha_detencion_aprehension"], errors="coerce")

    # hora como string limpio
    df_apre["hora_limpia"] = (
        df_apre["hora_detencion_aprehension"]
        .astype(str)
        .str.replace(" ", "")
        .str.strip()
    )

    # Combinar fecha + hora
    df_apre["fecha_completa"] = pd.to_datetime(
        df_apre["fecha"].astype(str) + " " + df_apre["hora_limpia"],
        errors="coerce"
    )

except Exception as e:
    print("Advertencia procesando fecha/hora:", e)



#estandarizar el codd parroquia
if "codigo_parroquia" in df_apre.columns:
    df_apre["codigo_parroquia"] = (
        df_apre["codigo_parroquia"]
        .astype(str)
        .str.replace(r"\.0$", "", regex=True)
        .str.zfill(6)
    )


#columnas de interes
cols_interes = [
    "fecha_completa", "fecha", "latitud", "longitud",
    "codigo_parroquia", "nombre_parroquia",
    "presunta_infraccion", "tipo", "arma", "movilizacion"
]

df_apre_clean = df_apre[cols_interes].copy()

# features temporales
df_apre_clean["franja_horaria"] = df_apre_clean["fecha_completa"].dt.hour
df_apre_clean["dia"] = df_apre_clean["fecha_completa"].dt.day
df_apre_clean["mes"] = df_apre_clean["fecha_completa"].dt.month
df_apre_clean["dia_semana"] = df_apre_clean["fecha_completa"].dt.dayofweek  # 0=Lunes

#grid espacial
df_apre_clean["lat_grid"] = df_apre_clean["latitud"].round(3)
df_apre_clean["lon_grid"] = df_apre_clean["longitud"].round(3)


#conteo de delios por dia y zona
grouped = (
    df_apre_clean
    .groupby(["lat_grid", "lon_grid", "fecha"])
    .size()
    .reset_index(name="conteo_delitos")
)

df_apre_clean = df_apre_clean.merge(
    grouped,
    on=["lat_grid", "lon_grid", "fecha"],
    how="left"
)


#guardar dataset limpio
print("\n=== Dataset final limpio ===")
print(df_apre_clean.head())
print(f"Total registros finales: {len(df_apre_clean)}")


Cargando archivo Excel... esto puede tardar unos segundos...
¡Cargado! Registros totales: 61149

=== Dataset final limpio ===
       fecha_completa      fecha   latitud   longitud codigo_parroquia  \
0 2025-02-03 08:00:00 2025-02-03 -3.715507 -79.618370           071150   
1 2025-05-16 13:40:00 2025-05-16 -1.046660 -77.742967           150155   
2 2025-05-20 13:30:00 2025-05-20 -2.260877 -79.878447           090150   
3 2025-04-11 17:00:00 2025-04-11 -3.270166 -79.953237           070150   
4 2025-09-18 15:30:00 2025-09-18 -0.951131 -79.358865           121150   

     nombre_parroquia                                presunta_infraccion  \
0           PORTOVELO                                            BOLETAS   
1   PUERTO MISAHUALLI                DELITOS CONTRA LOS RECURSOS MINEROS   
2  GUAYAQUIL-FLORESTA                                            BOLETAS   
3             MACHALA  DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...   
4            VALENCIA  DELITOS POR LA PRODUCCIÓN 

In [2]:


print("--- Primeras Filas---")
print(df_apre_clean.head())
print("\n--- Tipos de datos ---")
print(df_apre_clean.dtypes)

--- Primeras Filas---
       fecha_completa      fecha   latitud   longitud codigo_parroquia  \
0 2025-02-03 08:00:00 2025-02-03 -3.715507 -79.618370           071150   
1 2025-05-16 13:40:00 2025-05-16 -1.046660 -77.742967           150155   
2 2025-05-20 13:30:00 2025-05-20 -2.260877 -79.878447           090150   
3 2025-04-11 17:00:00 2025-04-11 -3.270166 -79.953237           070150   
4 2025-09-18 15:30:00 2025-09-18 -0.951131 -79.358865           121150   

     nombre_parroquia                                presunta_infraccion  \
0           PORTOVELO                                            BOLETAS   
1   PUERTO MISAHUALLI                DELITOS CONTRA LOS RECURSOS MINEROS   
2  GUAYAQUIL-FLORESTA                                            BOLETAS   
3             MACHALA  DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...   
4            VALENCIA  DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...   

          tipo     arma  movilizacion  franja_horaria  dia  mes  dia_semana 

In [3]:
# 2. Validación Geográfica Rápida
print("--- Rango de Coordenadas ---")
print(f"Latitud: {df_apre_clean['latitud'].min()} a {df_apre_clean['latitud'].max()}")
print(f"Longitud: {df_apre_clean['longitud'].min()} a {df_apre_clean['longitud'].max()}")


--- Rango de Coordenadas ---
Latitud: -4.977575151 a 2.087691416
Longitud: -90.983276367 a 79.913899269


In [4]:

# 3. Delitos más comunes
print("\n--- Top 10 Infracciones ---")
print(df_apre_clean['presunta_infraccion'].value_counts().head(10))


--- Top 10 Infracciones ---
presunta_infraccion
DELITOS CONTRA EL DERECHO A LA PROPIEDAD                                                         10727
BOLETAS                                                                                           9181
DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE SUSTANCIAS CATALOGADAS SUJETAS A FISCALIZACIÓN     9139
CONTRAVENCIONES DE TRÁNSITO                                                                       6080
DELITOS CONTRA LA SEGURIDAD PÚBLICA                                                               5177
CONTRAVENCIÓN DE VIOLENCIA CONTRA LA MUJER O MIEMBROS DEL NÚCLEO FAMILIAR                         3356
DELITOS DE VIOLENCIA CONTRA LA MUJER O MIEMBROS DEL NÚCLEO FAMILIAR                               2884
DELITOS CONTRA LA EFICIENCIA DE LA ADMINISTRACIÓN PÚBLICA                                         2245
CONTRAVENCIONES                                                                                   1546
DELITOS CULPOSOS DE TRÁN

In [5]:
# ============================================================
# CLASIFICACIÓN DE DELITOS GRAVES
# ============================================================

delitos_interes = [
    'DELITOS CONTRA EL DERECHO A LA PROPIEDAD',
    'DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE SUSTANCIAS CATALOGADAS SUJETAS A FISCALIZACIÓN',
    'DELITOS CONTRA LA SEGURIDAD PÚBLICA',
    'DELITOS CONTRA LA EFICIENCIA DE LA ADMINISTRACIÓN PÚBLICA',
    'DELITOS DE VIOLENCIA CONTRA LA MUJER O MIEMBROS DEL NÚCLEO FAMILIAR'
]

# 1. Etiqueta de delito grave (0/1)
df_apre_clean["es_delito_grave"] = df_apre_clean["presunta_infraccion"].isin(delitos_interes).astype(int)

# 2. Conteo solo de delitos graves (target alternativo)
df_graves = (
    df_apre_clean[df_apre_clean["es_delito_grave"] == 1]
    .groupby(["lat_grid", "lon_grid", "fecha"])
    .size()
    .reset_index(name="conteo_delitos_graves")
)

# Asegurar que no exista antes de unir
if "conteo_delitos_graves" in df_apre_clean.columns:
    df_apre_clean.drop(columns=["conteo_delitos_graves"], inplace=True)

# Merge limpio
df_apre_clean = df_apre_clean.merge(
    df_graves,
    on=["lat_grid", "lon_grid", "fecha"],
    how="left"
)

# Rellenar NaN con 0 (ningún delito grave en esa celda y día)
df_apre_clean["conteo_delitos_graves"] = df_apre_clean["conteo_delitos_graves"].fillna(0)

# ============================================================
# LIMPIEZA DE POSIBLES DUPLICADOS (col_x, col_y)
# ============================================================

cols_a_borrar = [c for c in df_apre_clean.columns if c.endswith("_x") or c.endswith("_y")]

if len(cols_a_borrar) > 0:
    print(f"\nEliminando columnas duplicadas generadas por merge: {cols_a_borrar}")
    df_apre_clean.drop(columns=cols_a_borrar, inplace=True)


print("\n--- Distribución de delitos graves (solo para análisis) ---")
print(df_apre_clean["es_delito_grave"].value_counts())


# ============================================================
# GUARDADO (solo CSV)
# ============================================================

df_apre_clean.to_csv("../data/interim/aprehendidos_Detenidos/aprehendidos_limpio_final.csv", index=False)

print("\n¡Archivo 'aprehendidos_limpio_final.csv' guardado con éxito!")
print(f"Registros finales: {len(df_apre_clean)}")



--- Distribución de delitos graves (solo para análisis) ---
es_delito_grave
0    30977
1    30172
Name: count, dtype: int64

¡Archivo 'aprehendidos_limpio_final.csv' guardado con éxito!
Registros finales: 61149
